In [10]:
import docker
client = docker.from_env()

In [11]:
for container in client.containers.list(all=True):
  print(container.name, container.status, container.image.tags, container.short_id)

container_9 running ['tf2:0.05'] df9dda980cd9
container_8 running ['tf2:0.05'] bd0ea5149273
container_7 running ['tf2:0.05'] 350b370d5587
container_6 running ['tf2:0.05'] 15cc52ec1ce0
container_5 running ['tf2:0.05'] f4e145827826
container_4 running ['tf2:0.05'] 22997f60caf9
container_3 running ['tf2:0.05'] 23eafc8fe174
container_2 running ['tf2:0.05'] ffbc0b74c319
container_1 running ['tf2:0.05'] 84b56c1edbeb
container_0 running ['tf2:0.05'] 781ffe5e0b64
nginx-nginx-1 running ['nginx:latest'] 319a7a894219


In [180]:
def get_current():
        current = 0
        for container in client.containers.list(all=True):
            if container.image.tags == ['tf2:0.05'] and container.status == 'running':
                current += 1
        return current
get_current()

0

In [49]:
client.api.remove_container("e1878e3058be", force=True)

In [7]:
container = client.containers.get('6e43d7a3976f1ec67c4073faca499f0a5fe31ca29cbafd24fbda4db586138f25')
print(container.stats(stream=True, decode=True))

<generator object APIClient._stream_helper at 0x7ff80169b140>


In [27]:
def start_container(number):
    exist = 0
    for gpu in range(4):
        for device in range (7):
            if (exist < number):
                container = client.containers.run(
                                                    'tf2:0.05', 
                                                    detach=True, 
                                                    ports={'8080/tcp': 8080+exist}, 
                                                    volumes={'/home/gil/Desktop/colab/': {'bind': '/home/user/workdir/data', 'mode': 'ro'}, 
                                                            '/home/gil/Desktop/alexander/code-server/user_dir': {'bind': '/home/user/workdir/example', 'mode': 'ro'},
                                                            '/etc/timezone': {'bind': '/etc/timezone', 'mode': 'ro'}, 
                                                            '/etc/localtime': {'bind': '/etc/localtime', 'mode': 'ro'},
                                                            '/home/gil/Desktop/alexander/code-server/config.yaml': {'bind': '/home/user/.config/code-server/config.yaml', 'mode': 'ro'}}, 
                                                    shm_size='1g', 
                                                    environment=['TZ=Asia/Seoul'], 
                                                    remove=False, 
                                                    name='container_'+str(exist),
                                                    device_requests=[{'DeviceIDs': [f'{gpu}:{device}'],
                                                                    'Capabilities': [['gpu']],
                                                                    }], 
                                                    command='code-server',
                                                    entrypoint='bash')
                exist += 1
                print("Run: " + container.short_id  + " gpu: " + str(gpu) + " device: " + str(device))

In [26]:
for container in client.containers.list():
  if container.image.tags == ['tf2:0.05']:
    container.remove(force=True)
    print("Removed container: ", container.short_id)

Removed container:  58cd6803986f3bb2aeec6f697c88e6f6591df290cfc4e42b4c88b19211b7f361
Removed container:  46d9bb81f84b963f2eac9d57fb9bf609d0c9d0f979421b863ad90be2ebc8b6a9
Removed container:  87ea50ddcd4ae75360003ad02874c83fd7fb9fa0d4381c599683f65b90e54a5a
Removed container:  0a677a5c62cd7fe04808543ddedd747d5e1e7c0e938303bc2b259170fe1178b3
Removed container:  dcfd6afbf43b8a165d910269a3e0c8946bebcb78d3f80ca5a9d26fff18dce322
Removed container:  3bc56de2308b0003b3d3dbe25c243ead304e609fe3b691ec09a28347cff15640
Removed container:  85f0faf8c0064e1821a50f2efc832bf4c8901183c0cc3c1861ca8ab7f4d14e77
Removed container:  02b2f3e8731ecb29a88ef7236147b053b12b9a2998eaec93276cd52b44021bf7
Removed container:  1917cbec6a491a80f1a9697181de1b30ca941ea83dc8d932feadc24065411e98
Removed container:  0750a4877611917ffa935b19a4e829d4aabc4b04f9b82cbcde330cfeee6f274d
Removed container:  1f0c89cd474cdbdf41fc334314276bfb03fde894d0e17b1fda6ef49f494a68a3
Removed container:  d41b8edde2ea7512599e8990366783e94b23d6951ab3a

In [28]:
start_container(28)

Run: 6deec6c04a49 gpu: 0 device: 0
Run: 07fc9064334e gpu: 0 device: 1
Run: d1188e3c96d2 gpu: 0 device: 2
Run: 31d952c14669 gpu: 0 device: 3
Run: 14c1ae70246a gpu: 0 device: 4
Run: 63329310c6be gpu: 0 device: 5
Run: f72d659c525d gpu: 0 device: 6
Run: a7d54cb7d4ce gpu: 1 device: 0
Run: a2bac594b398 gpu: 1 device: 1
Run: 3e91c150c3c3 gpu: 1 device: 2
Run: d9ac8dc0bbc3 gpu: 1 device: 3
Run: 535f77e4e3b3 gpu: 1 device: 4
Run: e9d108cb8fd1 gpu: 1 device: 5
Run: 11dbee07405f gpu: 1 device: 6
Run: eee0377cbe56 gpu: 2 device: 0
Run: 84c77f0018d8 gpu: 2 device: 1
Run: 007989c214c3 gpu: 2 device: 2
Run: 0454ac29a5f9 gpu: 2 device: 3
Run: 640e4aeb3cb5 gpu: 2 device: 4
Run: bbdeeeeb7219 gpu: 2 device: 5
Run: 9075b281ed87 gpu: 2 device: 6
Run: f2bf57419f62 gpu: 3 device: 0
Run: 2e183eadac83 gpu: 3 device: 1
Run: f7814977aae6 gpu: 3 device: 2
Run: 917c1ffc303f gpu: 3 device: 3
Run: 7567c78ff821 gpu: 3 device: 4
Run: ff9208197cca gpu: 3 device: 5
Run: 6248515b7522 gpu: 3 device: 6


In [175]:
def load_all_containers():
    for container in client.containers.list():
        if container.image.tags == ['tf2:0.05']:
            container.exec_run('conda run -n tf python /home/user/workdir/example/unet.py', detach=True)
load_all_containers()

In [182]:
def load_one(id):
    for container in client.containers.list():
        if container.short_id == id:
            container.exec_run('conda run -n tf python /home/user/workdir/example/unet.py', detach=True)
load_one("7567c78ff821")

In [162]:
import docker
from nvitop import MigDevice
client = docker.from_env()
mig = MigDevice.all()

def get_gpu_status():
     gpu_status = []
     for index, gpu in enumerate(mig):
          number = 27- index
          total = gpu.memory_total_human()
          free = gpu.memory_free_human()
          used = gpu.memory_used_human()
          gpu, device = gpu.index
          gpu_status.append(dict(index= number, gpu=gpu, device=device, total=total, free=free, used=used))
     return gpu_status
## python filter type to dict

def get_containers():
    containers = []
    gpu_status = get_gpu_status()
    for index, container in enumerate(client.containers.list(all=True)):
          if container.image.tags[0] in ["tf2:0.05", "tf1:0.01"]:
               cnt = dict(index=index, id=container.short_id, name=container.name, status=container.status)
               #gpu = list(filter(lambda x: x["index"] == index, gpu_status))
               my_dict = gpu_status[index]#gpu[0]
               containers.append({**cnt, **my_dict})
    return containers  

In [173]:
import time
st = time.process_time()
get_containers()
et = time.process_time()
res = et - st
print("--- %s seconds ---" % res)

--- 0.25064866799999663 seconds ---


In [174]:
get_containers()

[{'index': 27,
  'id': '6248515b7522',
  'name': 'container_27',
  'status': 'running',
  'gpu': 0,
  'device': 0,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 26,
  'id': 'ff9208197cca',
  'name': 'container_26',
  'status': 'running',
  'gpu': 0,
  'device': 1,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 25,
  'id': '7567c78ff821',
  'name': 'container_25',
  'status': 'running',
  'gpu': 0,
  'device': 2,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 24,
  'id': '917c1ffc303f',
  'name': 'container_24',
  'status': 'running',
  'gpu': 0,
  'device': 3,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 23,
  'id': 'f7814977aae6',
  'name': 'container_23',
  'status': 'running',
  'gpu': 0,
  'device': 4,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 22,
  'id': '2e183eadac83',
  'name': 'container_22',
  'status': 'running',
  'gpu': 0,
  'device': 5,


In [64]:
get_gpu_status()

[{'index': 27,
  'gpu': 0,
  'device': 0,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 26,
  'gpu': 0,
  'device': 1,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 25,
  'gpu': 0,
  'device': 2,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 24,
  'gpu': 0,
  'device': 3,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 23,
  'gpu': 0,
  'device': 4,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 22,
  'gpu': 0,
  'device': 5,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 21,
  'gpu': 0,
  'device': 6,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 20,
  'gpu': 1,
  'device': 0,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 19,
  'gpu': 1,
  'device': 1,
  'total': '9728MiB',
  'free': '9721MiB',
  'used': '7MiB'},
 {'index': 18,
  'gpu': 1,
  'device': 2,
  'total': '9728MiB',


In [71]:
get_containers()

ValueError: dictionary update sequence element #0 has length 6; 2 is required

In [8]:
from pynvml.smi import nvidia_smi
nvsmi = nvidia_smi.getInstance()
nvsmi.DeviceQuery('memory.free, memory.total')

{'gpu': [{'fb_memory_usage': {'total': 81920.0,
    'free': 81039.375,
    'unit': 'MiB'}},
  {'fb_memory_usage': {'total': 81920.0, 'free': 81039.375, 'unit': 'MiB'}},
  {'fb_memory_usage': {'total': 81920.0, 'free': 81039.375, 'unit': 'MiB'}},
  {'fb_memory_usage': {'total': 81920.0, 'free': 81039.375, 'unit': 'MiB'}}]}

In [10]:
from pynvml.smi import nvidia_smi
nvsmi = nvidia_smi.getInstance()
print(nvsmi.DeviceQuery('mig.mode.current'), end='\n')

{'gpu': [{'mig_mode': {'current_mm': 'Enabled', 'pending_mm': 'Enabled'}}, {'mig_mode': {'current_mm': 'Enabled', 'pending_mm': 'Enabled'}}, {'mig_mode': {'current_mm': 'Enabled', 'pending_mm': 'Enabled'}}, {'mig_mode': {'current_mm': 'Enabled', 'pending_mm': 'Enabled'}}]}


In [14]:
data = !nvidia-smi

In [15]:
print(data)

['Wed Mar 15 10:49:11 2023       ', '+-----------------------------------------------------------------------------+', '| NVIDIA-SMI 525.78.01    Driver Version: 525.78.01    CUDA Version: 12.0     |', '|-------------------------------+----------------------+----------------------+', '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |', '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |', '|                               |                      |               MIG M. |', '|===============================+======================+======================|', '|   0  NVIDIA A100 80G...  Off  | 00000000:17:00.0 Off |                   On |', '| N/A   33C    P0    46W / 300W |     45MiB / 81920MiB |     N/A      Default |', '|                               |                      |              Enabled |', '+-------------------------------+----------------------+----------------------+', '|   1  NVIDIA A100 80G...  Off  | 00000000:31:00.0

In [ ]:
import pynvml

pynvml.nvmlInit()
deviceCount = pynvml.nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    print("Device", i, ":", pynvml.nvmlDeviceGetName(handle))

for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    attributes = pynvml.nvmlDeviceGetAttributes(handle)
    print("Device", i, ":", pynvml.nvmlDeviceGetName(handle), " MIG enabled: ", bool(attributes.migEnabled))



In [ ]:
from nvitop import MigDevice
mig = MigDevice.all()


In [9]:
for i in mig:
    print(i.memory_total_human(), i.memory_free_human(), i.memory_used_human())

9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB
9728MiB 9721MiB 7MiB


In [14]:
pip install pymongo[srv]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 10.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 11.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient

mongodb_client = MongoClient('localhost', 27017)
database = mongodb_client['health']


In [9]:
database.address.find(limit)